In [1]:
# Import
import pandas as pd
import requests
import datetime
import plotly.express as px
import matplotlib
import numpy as np

# Pré-traitement 

In [2]:
# Charger csv
df_pro_res = pd.read_csv("/work/df_pro_res.csv", sep = ",")
df_meteo = pd.read_csv("/work/df_meteo.csv", sep = ",")
df_vacances_confinement = pd.read_csv("/work/vacances_confinement2.csv", sep = ",")
df_ferie = pd.read_csv("/work/jours_feries_metropole.csv", sep = ",")
df_ouvre_we_ferie = pd.read_csv("/work/jours-ouvres-week-end-feries-france-2010-a-2030.csv", sep=";")

In [3]:
# Groupby pro et résident
df_pro_res = df_pro_res.groupby(by = ["Profil_consommateur", "Date", "Région"]).agg({"Nb points soutirage" : "sum","Total énergie soutirée (Wh)" : "sum"}).reset_index()

In [4]:
# Calcul de la moyenne de la conso
df_pro_res["Consommation_moyenne"] = (df_pro_res["Total énergie soutirée (Wh)"]) / (df_pro_res["Nb points soutirage"])
df_pro_res.rename(columns = {"Date" : "DATE"}, inplace = True)

In [5]:
df_meteo.drop(["Code_région", "Ville", "TOTAL_SNOW_MM", "SUNSET", "SUNRISE","WEATHER_CODE_MORNING", "WEATHER_CODE_NOON", "WEATHER_CODE_EVENING", "OPINION"], axis = 1, inplace = True)
df_meteo = df_meteo.groupby(["DATE", "Région"]).mean().reset_index()

In [6]:
# Merge des deux DF
df_final = pd.merge(df_meteo, df_pro_res,
         how = "right",
         left_on = ["DATE", "Région"],
         right_on = ["DATE", "Région"]
         )

In [7]:
df_vacances_confinement.rename(columns = {"Date" : "DATE"}, inplace = True)

In [8]:
df_final = pd.merge(df_vacances_confinement, df_final,
                    how = "right",
                    left_on = "DATE",
                    right_on = "DATE")

In [9]:
df_ouvre_we_ferie.rename(columns = {"Date" : "DATE"}, inplace = True)

In [10]:
df_final = pd.merge(df_ouvre_we_ferie, df_final,
                    how = "right",
                    left_on = "DATE",
                    right_on = "DATE")

In [11]:
df_final.drop(["Jour et date", "Description_x"], axis =1, inplace = True)

In [12]:
# Sauvegarde
df_final.to_csv("/work/df_final.csv", index = False)

# Machine Learning
Pour les résidents :
Ne pas prendre Juin à Septembre car plateau de consommation. En effet, dès que la température dépasse les environ 14°C de moyenne de moyenne, on arrive sur un plateau de consommation qui est inférieur à 10 000 Wh.

On ne prend que les variables de température.
et pluie ??

Algo de régression linéaire ?

Pour les professionnels :
Autre algo de régression linéaire


## Chargement

In [13]:
df_final = pd.read_csv("/work/df_final.csv", sep=",")

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor

In [15]:
df_final = pd.get_dummies(df_final, columns=['Statut', 'Description_y', 'Région', 'Profil_consommateur'])

In [16]:
df_final_20_22 = df_final[df_final["DATE"].between("2020-01-01", "2022-12-31")]

In [17]:
df_final_20_22["DATE"] = pd.to_datetime(df_final_20_22["DATE"])

/tmp/ipykernel_730/3792895100.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_20_22["DATE"] = pd.to_datetime(df_final_20_22["DATE"])


In [18]:
df_final_20_22["Mois"] = df_final_20_22["DATE"].dt.month

/tmp/ipykernel_730/280383646.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_20_22["Mois"] = df_final_20_22["DATE"].dt.month


In [19]:
# Sauvegarde
df_final_20_22.to_csv("/work/df_final_20_22.csv", index = False)

In [20]:
# On défini nos variables X et y:
y = df_final_20_22["Consommation_moyenne"]

# mettre année, mois
X = df_final_20_22.select_dtypes(include='number').drop(["Consommation_moyenne", "Nb points soutirage", "Total énergie soutirée (Wh)"], axis = 1)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle = False)

## Test des algorithmes

In [22]:
liste_model_test = [LinearRegression(), RandomForestRegressor()]

liste_scaler_test = [MinMaxScaler(), StandardScaler()]


for model in liste_model_test:
    for scaler in liste_scaler_test:
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)

        print(type(model).__name__,type(scaler).__name__,"rmse =", mean_squared_error(y_test, y_pred, squared=False))
        print("score =", r2_score(y_test, y_pred))

LinearRegression MinMaxScaler rmse = 3238.4875641431463
score = 0.6142913177869955
LinearRegression StandardScaler rmse = 3239.4229815604504
score = 0.6140684663976093
RandomForestRegressor MinMaxScaler rmse = 1601.020973650474
score = 0.9057309804862081
RandomForestRegressor StandardScaler rmse = 1616.7461056302707
score = 0.9038700769947068


On choisit :  RandomForestRegressor MinMaxScaler rmse = 1744.655711420967
score = 0.9574541946966378

In [23]:
# Scalérisation du modèle + randomforestregressor
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = RandomForestRegressor()

model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)


# Évaluer les performances du modèle
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print ('rmse :', rmse)
print('r2:', r2)


rmse : 1620.568975597762
r2: 0.9034149323428795


## Analyse erreur

In [24]:
y_pred = model.predict(X_test)

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [25]:
y_test.median()

9815.903557016643

In [26]:
y_test.mean()

12620.808803082928

In [27]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE:", rmse)

RMSE: 5197.072191715839


In [28]:
r2 = r2_score(y_test, y_pred)

# Affichage du coefficient de détermination (R²)
print("R²:", r2)

R²: 0.0066725282297736


In [29]:
print(mean_absolute_percentage_error(y_test, y_pred)*100)

33.67210815237024


In [30]:
def median_absolute_percentage_error(y_true, y_predict):
    return np.median((np.abs(np.subtract(y_true, y_predict) / y_true)))

In [31]:
print(median_absolute_percentage_error(y_test, y_pred)*100)

37.39217874493711


## Gridsearch

In [32]:
from sklearn.model_selection import GridSearchCV

In [33]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

In [34]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, verbose = 10)
# verbose permet d'afficher l'avancement

In [35]:
grid_search.fit(X_train_scaled, y_train)

[CV 2/5; 90/162] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=300;, score=-9326930.740 total time=   0.7s
[CV 3/5; 90/162] START max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=300
[CV 3/5; 90/162] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=300;, score=-12585754.717 total time=   0.7s
[CV 4/5; 90/162] START max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=300
[CV 4/5; 90/162] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=300;, score=-6992965.039 total time=   0.7s
[CV 5/5; 90/162] START max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=300
[CV 5/5; 90/162] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=300;, score=-14170276.270 total time=   0.7s
[CV 1/5; 91/162] START max_depth=5, max_features

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [None, 5, 10],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_squared_error', verbose=10)

In [41]:
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# faire la racine carrée  et mettre en valeur absolue
abs_best_score = abs(best_score)
from math import sqrt
sqrt_x = sqrt(abs_best_score)


print("Best paramètres :", best_params)
print("Best score :", sqrt_x) 

Best paramètres : {'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best score : 2207.259724563637


In [39]:
model = RandomForestRegressor(max_depth=None, max_features="auto", min_samples_leaf=1, min_samples_split=2, n_estimators=100)

model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

# Évaluer les performances du modèle
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print ('rmse :', rmse)
print('r2 :', r2)

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
rmse : 1602.7161007455081
r2 : 0.9055312547272282


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=136c2633-d840-4047-9a79-ab6b0fb04232' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>